In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data from an Excel file
data = pd.read_excel('Wind_Energy_Rayalaseema.xlsx')
data.head()



,date_hourly,hourly,soil_temp1,temp1,solar_rd,u10,v10,soil_temp,temp,pressure,wind_speed,Wind_Energy
0,2011-01-01,1.0,293.431,292.019,1.961410e+07,0.155315,1.399950,20.281,18.869,95848.4,1.408539,1.710245
1,2011-01-01,2.0,293.063,291.975,1.862650e-09,0.114499,1.303050,19.913,18.825,95901.0,1.308071,1.369762
2,2011-01-01,3.0,293.102,292.538,1.086020e+05,-0.272937,1.208260,19.952,19.388,95997.9,1.238704,1.163198
3,2011-01-01,4.0,295.555,295.004,8.415560e+05,-0.401350,1.049350,22.405,21.854,96101.7,1.123484,0.867866
4,2011-01-01,5.0,299.023,297.310,2.294920e+06,-0.516890,0.721392,25.873,24.160,96141.0,0.887458,0.427755


In [2]:
# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:")
print(missing_values)

# Remove rows with missing values (if desired)
data = data.dropna()

# Check for and remove duplicates (if desired)
data = data.drop_duplicates()

# Outlier detection and handling (you can customize this)
from scipy import stats
z_scores = np.abs(stats.zscore(data[['soil_temp', 'temp', 'pressure', 'wind_speed']]))
threshold = 3
outliers = (z_scores > threshold).all(axis=1)
data = data[~outliers]

# Data standardization (if not already done)
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Now, you can proceed with splitting the cleaned data and training the model.


Missing Values:
 date_hourly        0
hourly          95593
soil_temp1          1
temp1               0
solar_rd            0
u10                 0
v10              1105
soil_temp           1
temp                1
pressure         1105
wind_speed          0
Wind_Energy         0
dtype: int64


In [3]:
# Extract features (X) and target (y) from the cleaned dataset
X = data[['soil_temp', 'temp', 'pressure', 'wind_speed']].values
y = data['Wind_Energy'].values.reshape(-1, 1)

# Split the cleaned data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (if not already done)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Now you can proceed with building and training your model using the cleaned and split data.

In [4]:
# Extract features (X) and target (y) from the dataset
X = data[['soil_temp', 'temp', 'pressure', 'wind_speed']].values
y = data['Wind_Energy'].values.reshape(-1, 1)

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
# Build the ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with a single neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')

# Make predictions
predictions = model.predict(X_test)

# You can use 'predictions' to predict wind energy based on new input data.

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 695.6073 - mae: 20.9034 - val_loss: 470.0452 - val_mae: 16.7644
Epoch 2/200
1/1 [==============================] - 0s 45ms/step - loss: 694.5698 - mae: 20.8813 - val_loss: 469.3152 - val_mae: 16.7440
Epoch 3/200
1/1 [==============================] - 0s 40ms/step - loss: 693.5358 - mae: 20.8594 - val_loss: 468.5914 - val_mae: 16.7237
Epoch 4/200
1/1 [==============================] - 0s 42ms/step - loss: 692.4955 - mae: 20.8373 - val_loss: 467.8614 - val_mae: 16.7034
Epoch 5/200
1/1 [==============================] - 0s 40ms/step - loss: 691.4468 - mae: 20.8149 - val_loss: 467.1277 - val_mae: 16.6831
Epoch 6/200
1/1 [==============================] - 0s 40ms/step - loss: 690.3954 - mae: 20.7924 - val_loss: 466.3842 - val_mae: 16.6626
Epoch 7/200
1/1 [==============================] - 0s 42ms/step - loss: 689.3362 - mae: 20.7696 - val_loss: 465.6393 - val_mae: 16.6420
Epoch 8/200
1/1 [==============================] -

In [8]:
# Save the trained model to a file
model.save('wind_energy_prediction_model.h5')

# Load the saved model (if needed)
loaded_model = tf.keras.models.load_model('wind_energy_prediction_model.h5')
new_soil_temp = 25.0
new_temp = 20.0
new_pressure = 95848
new_wind_speed = 1.4

# Make predictions using new input data
new_input_data = np.array([[new_soil_temp, new_temp, new_pressure, new_wind_speed]])  # Replace with your new data
new_input_data = scaler.transform(new_input_data)  # Standardize the new input data

# Predict wind energy
predicted_wind_energy = model.predict(new_input_data)

print(f'Predicted Wind Energy: {predicted_wind_energy[0][0]}')


c:\Users\Subhang Mokkarala\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 32ms/step
Predicted Wind Energy: 1.613121509552002
